** Preprocessing **


### Imports and requirements

* Мы имеем дело с последовательностями, один из интуитивных способов работы с ними - использование рекуррентных сетей. Преимущество нейронных сетей заключается в том, что можно строить хорошие решения без использования сложного и трудоемкого feature engineering-а (чтобы эффективно решать ту же задачу с высоким качеством с помощью бустингов нужно несколько тысяч признаков), благодаря рекуррентным сетям. 

In [35]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

pd.set_option('display.max_columns', 80)

# добавим папки для импорта модулей, в них лежат все необходимые полезные функции для обработки данных
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
TRAIN_TRANSACTIONS_PATH = '/media/DATA/AlfaBattle/train_transactions_contest/'
TEST_TRANSACTIONS_PATH = '/media/DATA/AlfaBattle/test_transactions_contest/'

TRAIN_TARGET_PATH = '/media/DATA/AlfaBattle/train_target.csv'
PRE_TRANSACTIONS_PATH = '/media/DATA/AlfaBattle//preprocessed_transactions/'
PRE_TEST_TRANSACTIONS_PATH = '/media/DATA/AlfaBattle/preprocessed_test_transactions/'
PICKLE_VAL_BUCKET_PATH = '/media/DATA/AlfaBattle/val_buckets/'
PICKLE_VAL_TRAIN_BUCKET_PATH = '/media/DATA/AlfaBattle/val_train_buckets/'
PICKLE_VAL_TEST_BUCKET_PATH = '/media/DATA/AlfaBattle/val_test_buckets/'

In [37]:
target_frame = pd.read_csv(TRAIN_TARGET_PATH)
target_frame.head()

,app_id,product,flag
0,0,3,0
1,1,1,0
2,2,1,0
3,3,1,0
4,4,1,0


* Мы не можем поместить всю выборку в память, в виду, например, ограниченных ресурсов. Для итеративного чтения данных нам потребуется функция `utils.read_parquet_dataset_from_local`, которая читает N частей датасета за раз в память.


* Нейронные сети требуют отдельного внимания к тому, как будут поданы и обработаны данные. Важные моменты, на которые требуется обратить внимание:  

* Использование рекуррентных сетей подразумевает работу на уровне последовательностей, где одна последовательность - все исторические транзакции клиента. Для преобразования  `pd.DataFrame` с транзакциями клиентов в табличном виде к последовательностям используется функция `dataset_preprocessing_utils.transform_transactions_to_sequences`, она делает необходимые манипуляции и возвращает фрейм с двумя колонками: `app_id` и `sequences`. Колонка `sequence` представляет из себя массив массивов длины `len(features)`, где каждый вложенный массив - значения одного конкретного признака во всех транзакциях клиента. 
    
* каждый клиент имеет различную по длине историю транзакций. При этом обучение сетей происходит батчами, что требует делать паддинги в последовательностях. Довольно неэффективно делать паддинг внутри батча на последовательностях случайной длины (довольно часто будем делать большой и бесполезный паддинг). Гораздо лучше использовать технику `sequence_bucketing`. Эту операцию выполняет функция `dataset_preprocessing_utils.create_padded_buckets`. Один из аргументов в данную функцию - `bucket_info` - словарь, где для конкретной длины последовательности указано до какой длины нужно делать паддинг. Используется простой вид разбиения на 100 бакетов и файл где лежит отображение каждой длины в паддинг (файл `buckets_info.pkl`).
    
* Такие признаки, как [`amnt`, `days_before`, `hour_diff`] по своей природе не являются категориальными. Но можно интерпретировать каждую не категориальную фичу как категориальную. Для этого нужно подготовить bin-ы для каждой фичи. 

In [38]:
from utils import read_parquet_dataset_from_local
from dataset_preprocessing_utils import transform_transactions_to_sequences, create_padded_buckets

In [39]:
import pickle

with open('../constants/buckets_info.pkl', 'rb') as f:
    mapping_seq_len_to_padded_len = pickle.load(f)
    
with open('../constants/dense_features_buckets.pkl', 'rb') as f:
    dense_features_buckets = pickle.load(f)

### Preprocessing

In [ ]:
* Функция `create_buckets_from_transactions` ниже реализует следующий набор действий:
    * Читает `num_parts_to_preprocess_at_once` частей датасета в память
    * Преобразует вещественные и численные признаки к категориальным (используя `np.digitize` и подготовленные бины)
    * Формирует фрейм с транзакциями в виде последовательностей с помощью `transform_transactions_to_sequences`.
    * Если указан `frame_with_ids`, то использует `app_id` из `frame_with_ids` - актуально, чтобы выделить валидационную выборку.
    * Реализует технику `sequence_bucketing` и сохраняет словарь обработанных последовательностей в `.pkl` файл

In [40]:
def create_buckets_from_transactions(path_to_dataset, save_to_path, frame_with_ids = None, 
                                     num_parts_to_preprocess_at_once: int = 1, 
                                     num_parts_total=50, has_target=False):
    block = 0
    for step in tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once), 
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once, 
                                                             verbose=True)
        for dense_col in ['amnt', 'days_before', 'hour_diff']:
            transactions_frame[dense_col] = np.digitize(transactions_frame[dense_col], bins=dense_features_buckets[dense_col])
            
        seq = transform_transactions_to_sequences(transactions_frame)
        seq['sequence_length'] = seq.sequences.apply(lambda x: len(x[1]))
        
        if frame_with_ids is not None:
            seq = seq.merge(frame_with_ids, on='app_id')

        block_as_str = str(block)
        if len(block_as_str) == 1:
            block_as_str = '00' + block_as_str
        else:
            block_as_str = '0' + block_as_str
            
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=has_target, 
                                                    save_to_file_path=os.path.join(save_to_path, 
                                                                                   f'processed_chunk_{block_as_str}.pkl'))
        block += 1

In [41]:
train, val = train_test_split(target_frame, random_state=42, test_size=0.1)
train.shape, val.shape

((867429, 3), (96382, 3))

In [42]:
! rm -r {PICKLE_VAL_BUCKET_PATH}
! mkdir {PICKLE_VAL_BUCKET_PATH}

In [43]:
create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH, 
                                save_to_path=PICKLE_VAL_BUCKET_PATH,
                                frame_with_ids=val, num_parts_to_preprocess_at_once=5, num_parts_total=50, has_target=True)

Transforming transactions data:   0%|          | 0/10 [00:00<?, ?it/s]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_000_0_to_23646.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_001_23647_to_47415.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_002_47416_to_70092.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_003_70093_to_92989.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_004_92990_to_115175.parquet


Extracting buckets:   0%|          | 0/100 [00:00<?, ?it/s]

Transforming transactions data:  10%|█         | 1/10 [00:38<05:43, 38.14s/it]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_005_115176_to_138067.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_006_138068_to_159724.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_007_159725_to_180735.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_008_180736_to_202834.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_009_202835_to_224283.parquet


Extracting buckets:   0%|          | 0/100 [00:00<?, ?it/s]

Transforming transactions data:  20%|██        | 2/10 [01:16<05:06, 38.28s/it]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_010_224284_to_245233.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_011_245234_to_265281.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_012_265282_to_285632.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_013_285633_to_306877.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_014_306878_to_329680.parquet


...

Transforming transactions data:  80%|████████  | 8/10 [05:09<01:17, 38.70s/it]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_040_823300_to_841218.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_041_841219_to_859270.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_042_859271_to_878521.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_043_878522_to_896669.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_044_896670_to_916056.parquet


Extracting buckets:   0%|          | 0/100 [00:00<?, ?it/s]

Transforming transactions data:  90%|█████████ | 9/10 [05:50<00:39, 39.50s/it]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_045_916057_to_935131.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_046_935132_to_951695.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_047_951696_to_970383.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_048_970384_to_987313.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_049_987314_to_1003050.parquet


Extracting buckets:   0%|          | 0/100 [00:00<?, ?it/s]

Transforming transactions data: 100%|██████████| 10/10 [06:24<00:00, 38.45s/it]


In [44]:
path_to_dataset = PICKLE_VAL_BUCKET_PATH
dir_with_datasets = os.listdir(path_to_dataset)
dataset_val = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_val

['/media/DATA/AlfaBattle/val_buckets/processed_chunk_000.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_001.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_002.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_003.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_004.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_005.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_006.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_007.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_008.pkl',
 '/media/DATA/AlfaBattle/val_buckets/processed_chunk_009.pkl']

In [45]:
! rm -r {PICKLE_VAL_TRAIN_BUCKET_PATH}
! mkdir {PICKLE_VAL_TRAIN_BUCKET_PATH}

In [46]:
create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH, 
                                save_to_path=PICKLE_VAL_TRAIN_BUCKET_PATH,
                                frame_with_ids=train, num_parts_to_preprocess_at_once=5, num_parts_total=50, has_target=True)

Transforming transactions data:   0%|          | 0/10 [00:00<?, ?it/s]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_000_0_to_23646.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_001_23647_to_47415.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_002_47416_to_70092.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_003_70093_to_92989.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_004_92990_to_115175.parquet


Extracting buckets:   0%|          | 0/100 [00:00<?, ?it/s]

Transforming transactions data:  10%|█         | 1/10 [01:00<09:08, 61.00s/it]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_005_115176_to_138067.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_006_138068_to_159724.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_007_159725_to_180735.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_008_180736_to_202834.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_009_202835_to_224283.parquet


Extracting buckets:   0%|          | 0/100 [00:00<?, ?it/s]

Transforming transactions data:  20%|██        | 2/10 [02:00<08:03, 60.38s/it]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_010_224284_to_245233.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_011_245234_to_265281.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_012_265282_to_285632.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_013_285633_to_306877.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_014_306878_to_329680.parquet


...

Transforming transactions data:  80%|████████  | 8/10 [07:39<01:53, 56.75s/it]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_040_823300_to_841218.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_041_841219_to_859270.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_042_859271_to_878521.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_043_878522_to_896669.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_044_896670_to_916056.parquet


Extracting buckets:   0%|          | 0/100 [00:00<?, ?it/s]

Transforming transactions data:  90%|█████████ | 9/10 [08:48<01:00, 60.65s/it]Reading chunks:

/media/DATA/AlfaBattle/train_transactions_contest/part_045_916057_to_935131.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_046_935132_to_951695.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_047_951696_to_970383.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_048_970384_to_987313.parquet
/media/DATA/AlfaBattle/train_transactions_contest/part_049_987314_to_1003050.parquet


Extracting buckets:   0%|          | 0/100 [00:00<?, ?it/s]

Transforming transactions data: 100%|██████████| 10/10 [09:46<00:00, 58.66s/it]


In [47]:
path_to_dataset = PICKLE_VAL_TRAIN_BUCKET_PATH
dir_with_datasets = os.listdir(path_to_dataset)
dataset_train = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_train

['/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_000.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_001.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_002.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_003.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_004.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_005.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_006.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_007.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_008.pkl',
 '/media/DATA/AlfaBattle/val_train_buckets/processed_chunk_009.pkl']